In [39]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from keras.layers import Dense, Flatten, Conv2D

%matplotlib inline

In [40]:
from keras.datasets import cifar10

In [41]:
(X_tr, Y_tr),(X_te, Y_te) = cifar10.load_data()
X_tr.shape, Y_tr.shape, X_te.shape, Y_te.shape

((50000, 32, 32, 3), (50000, 1), (10000, 32, 32, 3), (10000, 1))

In [42]:
from keras.layers import MaxPooling2D, Dropout, Activation
from keras.models import Sequential
from keras.utils import to_categorical

Y_ = np.reshape(Y_tr, (-1))
n_classes = len(set(Y_))



n_classes

10

In [43]:
y_train = to_categorical(Y_tr, n_classes)
y_te = to_categorical(Y_te, n_classes)

In [47]:
def CNN():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=X_tr.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))          
    model.add(Activation('relu'))
    #model.add(MaxPooling2D(pool_size=(2,2)))
    #model.add(Dropout(0.5))
    
    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))          
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    #model.add(Dropout(0.5))
    
    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(128, (3, 3)))          
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    #model.add(Dropout(0.5))
    
    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(128, (3, 3)))          
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    
    
    # Fully Conected 
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(n_classes))
    model.add(Activation('softmax'))          
              
    return model          
    

In [48]:
model = CNN()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
activation_21 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_22 (Activation)   (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 30, 30, 64)        18496     
_________________________________________________________________
activation_23 (Activation)   (None, 30, 30, 64)        0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 28, 28, 64)        36928     
__________

In [49]:
from keras.optimizers import Adam
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [50]:
from keras.preprocessing.image import ImageDataGenerator
gen = ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                        height_shift_range=0.08, zoom_range=0.08)

In [51]:
test_gen = ImageDataGenerator()

In [52]:
batch_size = 64


In [53]:
train_generator = gen.flow(X_tr, y_train, batch_size=batch_size)
test_generator = test_gen.flow(X_te, y_te, batch_size=batch_size)

In [ ]:
model.fit_generator(train_generator, steps_per_epoch=X_tr.shape[0]//batch_size, epochs=10,
                   validation_data=test_generator, validation_steps=X_te.shape[0]//batch_size)

Epoch 1/10
781/781 [==============================] - 842s 1s/step - loss: 1.8593 - acc: 0.3121 - val_loss: 1.4600 - val_acc: 0.4599
Epoch 2/10
780/781 [============================>.] - ETA: 1s - loss: 1.4199 - acc: 0.4830

In [ ]:
test = model.evaluate(X_te, y_te)
print('Testing results:{}'.format(score[1]))